In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from datetime import datetime, timedelta



In [2]:
def get_table_data(driver, date):
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
    table = driver.find_element(By.TAG_NAME, 'table')
    rows = table.find_elements(By.TAG_NAME, 'tr')
    data = []
    for row in rows[1:]:  # Skipping header row
        cols = row.find_elements(By.TAG_NAME, 'td')
        cols = [col.text for col in cols]
        cols.append(date)  # Adding the date as the last column
        data.append(cols)
    return data


def set_date_filter(driver, date):
    date_input = driver.find_element(By.ID, 'datetma')
    date_input.clear()
    date_input.send_keys(date)
    date_input.send_keys(Keys.RETURN)
    filter_button = driver.find_element(By.CLASS_NAME, 'mybutton1')
    filter_button.click()

def scrape_data_for_date(driver, date):
    try:
        set_date_filter(driver, date)
        data = get_table_data(driver,date)
        return data
    except TimeoutException as e:
        print(f"TimeoutException: The table was not found for date {date}.")
        return []

def scrape_data_with_selenium(start_date, end_date):
    driver = webdriver.Chrome()
    driver.get("https://bpbd.jakarta.go.id/waterlevel")
    
    all_data = []
    current_date = start_date

    while current_date <= end_date:
        formatted_date = current_date.strftime("%d/%m/%Y")
        print(f"Scraping data for date: {formatted_date}")
        data = scrape_data_for_date(driver, formatted_date)
        all_data.extend(data)
        current_date += timedelta(days=1)

    driver.quit()
    return all_data

# Define the start and end dates for a year
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 6, 30)

# Scrape the data
all_data = scrape_data_with_selenium(start_date, end_date)

# Print the first few rows to understand the structure
print(all_data[:5])


Scraping data for date: 01/01/2023
Scraping data for date: 02/01/2023
Scraping data for date: 03/01/2023
Scraping data for date: 04/01/2023
Scraping data for date: 05/01/2023
Scraping data for date: 06/01/2023
Scraping data for date: 07/01/2023
Scraping data for date: 08/01/2023
Scraping data for date: 09/01/2023
Scraping data for date: 10/01/2023
Scraping data for date: 11/01/2023
Scraping data for date: 12/01/2023
Scraping data for date: 13/01/2023
Scraping data for date: 14/01/2023
Scraping data for date: 15/01/2023
Scraping data for date: 16/01/2023
Scraping data for date: 17/01/2023
Scraping data for date: 18/01/2023
Scraping data for date: 19/01/2023
Scraping data for date: 20/01/2023
Scraping data for date: 21/01/2023
Scraping data for date: 22/01/2023
Scraping data for date: 23/01/2023
Scraping data for date: 24/01/2023
Scraping data for date: 25/01/2023
Scraping data for date: 26/01/2023
Scraping data for date: 27/01/2023
Scraping data for date: 28/01/2023
Scraping data for da

In [3]:
# Adjust the column names based on the actual table structure
columns = [
    'Pintu Air', '23:00', '22:00', '21:00', '20:00', '19:00',
    '18:00', '17:00', '16:00', '15:00', '14:00', '13:00',
    '12:00', '11:00', '10:00', '09:00', '08:00', '07:00',
    '06:00', '05:00', '04:00', '03:00', '02:00', '01:00','00:00','add','add','add','add','add','add','Tanggal'
]  # Adjust these names as necessary

# Create a DataFrame
df = pd.DataFrame(all_data, columns=columns)

# Save to CSV or any other format
df.to_excel('Scrap2324.xlsx', index=False)
print("Data scraping completed and saved to 'water_level_data.csv'")


Data scraping completed and saved to 'water_level_data.csv'


In [4]:
# Menghilangkan baris 2, 15, 28, dst.
df = df.drop(index=df.index[0::13])

df.to_excel('TMA20232024.xlsx', index=False)

In [27]:
import pandas as pd
import numpy as np

# # Load the Excel file
# file_path = 'TMA2023.xlsx'  # ganti dengan path file yang sesuai
# df = pd.read_excel(file_path)

# Prepare the date range and time values
dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
times = [f'{hour:02d}:00' for hour in range(24)]

# Prepare the new DataFrame structure
num_dates = len(dates)
num_times = len(times)
num_pintu_air = 12
total_rows = num_dates * num_times

# Initialize the new DataFrame
columns = ['Tanggal', 'Waktu'] + [f'Pintu Air {i+1}' for i in range(num_pintu_air)]
new_df = pd.DataFrame(columns=columns)

# Populate the Tanggal and Waktu columns
new_df['Tanggal'] = np.repeat(dates, num_times)
new_df['Waktu'] = times * num_dates

# Extract the measurements from the original DataFrame
# Dropping the first row as it is a duplicate header
original_data = df.iloc[1:, 1:].reset_index(drop=True)
pintu_air_columns = original_data.columns[:num_pintu_air].tolist()

# Repeat the original data for the entire year
repeated_data = np.tile(original_data[pintu_air_columns].values, (num_dates, 1))

# Adjust the size to fit exactly into the new DataFrame
adjusted_data = repeated_data[:total_rows, :num_pintu_air]

# Assign the adjusted data to the new DataFrame
for i, col in enumerate(new_df.columns[2:]):
    new_df[col] = adjusted_data[:, i]

# Save the new DataFrame to a new Excel file
output_file_path = 'Reformatted_TMA2023.xlsx'  # ganti dengan path file output yang diinginkan
new_df.to_excel(output_file_path, index=False)

print(f'File telah disimpan di: {output_file_path}')


File telah disimpan di: Reformatted_TMA2023.xlsx


In [23]:
import pandas as pd

df = pd.read_excel("TMA20232024.xlsx")

# Jumlah baris per blok
n = 12

# List untuk menyimpan blok yang ditranspos
transposed_blocks = []

# Memproses setiap blok 13 baris
for i in range(0, len(df), n):
    # Memilih blok 13 baris
    block = df.iloc[i:i+n]
    
    # Mentranpos blok
    transposed_block = block.T
    
    # Menyimpan blok yang ditranspos
    transposed_blocks.append(transposed_block)

# Menggabungkan semua blok yang ditranspos ke bawah
result = pd.concat(transposed_blocks)
#result = transposed_df.iloc[:,0:12]

# Mengatur ulang indeks
#result.reset_index(drop=True, inplace=True)

print(result.iloc[:,0:12].info())


# # Reset index agar index dari 0 hingga akhir
# transposed_df.reset_index(drop=True, inplace=True)

result.to_excel("HasilTransformasi-Baru.xlsx", index=True)
print("Selesai Ditransformasi!")

<class 'pandas.core.frame.DataFrame'>
Index: 17504 entries, Pintu Air to Tanggal
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       27 non-null     object
 1   1       27 non-null     object
 2   2       27 non-null     object
 3   3       27 non-null     object
 4   4       27 non-null     object
 5   5       27 non-null     object
 6   6       27 non-null     object
 7   7       27 non-null     object
 8   8       27 non-null     object
 9   9       27 non-null     object
 10  10      27 non-null     object
 11  11      27 non-null     object
dtypes: object(12)
memory usage: 1.7+ MB
None
Selesai Ditransformasi!


In [11]:
print(transposed_df.iloc[:,0:12].head(5))

                  0           1              2           3                4   \
0  Bendung Katulampa   Pos Depok  Manggarai BKB   PA. Karet  Pos Krukut Hulu   
1         40 cm (MT)  115 cm (T)     710 cm (T)  350 cm (T)        60 cm (T)   
2         40 cm (MT)  115 cm (T)     715 cm (T)  350 cm (T)        60 cm (T)   
3         40 cm (MT)  115 cm (T)     715 cm (T)  350 cm (T)        60 cm (T)   
4         40 cm (MT)  120 cm (T)     710 cm (T)  340 cm (T)        60 cm (T)   

                 5               6            7                  8   \
0  Pos Pesanggrahan  Pos Angke Hulu  Waduk Pluit  Pasar Ikan - Laut   
1         90 cm (T)      120 cm (T)  -175 cm (T)         155 cm (M)   
2         90 cm (T)      120 cm (T)  -175 cm (T)         150 cm (T)   
3         90 cm (T)      120 cm (T)  -160 cm (T)         147 cm (T)   
4         90 cm (T)      120 cm (T)  -160 cm (M)         150 cm (T)   

                  9                10           11  
0  Pos Cipinang Hulu  Pos Sunter Hulu  